In [ ]:
import numpy as np
from prepare_data import get_data
from keras import models, layers, optimizers, backend


(train_X_ims, train_X_seqs, train_Y, test_X_ims, test_X_seqs,
            test_Y, im_shape, vocab_size, num_answers,
            all_answers, test_qs, test_answer_indices) = get_data()


image_input = layers.Input(shape=im_shape)
image_layers = layers.Conv2D(8, 3, padding='same')(image_input)
image_layers = layers.MaxPooling2D()(image_layers)
image_layers = layers.Conv2D(16, 3, padding='same')(image_layers)
image_layers = layers.MaxPooling2D()(image_layers)
image_layers = layers.Flatten()(image_layers)
image_layers = layers.Dense(32, activation='tanh')(image_layers)

# The question networks
q_input = layers.Input(shape=(train_X_seqs.shape[-1],))

q_layers = layers.Dense(32, activation='tanh')(q_input)
q_layers = layers.Dense(32, activation='tanh')(q_layers)
q_layers = layers.Dense(1, activation='tanh')(q_layers)

# Merge -> output
out = layers.Multiply()([image_layers, q_layers])
out = layers.Embedding(input_dim=vocab_size, output_dim=60)(out)
out = layers.Dense(32, activation='tanh')(out)
out = layers.Flatten()(out)
out = layers.Dense(num_answers, activation='softmax')(out)

model = models.Model(inputs=[image_input, q_input], outputs=out)
model.compile(optimizers.Adam(lr=5e-4), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model_history = model.fit([train_X_ims, train_X_seqs], train_Y,
                        epochs=10,
                        batch_size=128,
                        verbose=2,
                        shuffle='batch',
                        validation_split=0.2)

In [ ]:
model.summary()

In [ ]:
layer_index: int = 11
get_embedding_output = backend.function([model.layers[layer_index].input],
                                  [model.layers[layer_index].output])
embedding_output = get_embedding_output([input_data])[0]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

X_train = input_data
y_train = model.predict(input_data)
knn = KNeighborsClassifier(n_neighbors=k,
                           weights='distance').fit(X_train, y_train)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def knn_performance(X_train, y_train, X_test, y_test, k=4, preds_file=''):
    '''
    Given a train and test split, measure the overall accuracy,
    precision, recall, F-1 score and support of the kNN classifier.
    '''
    knn = KNeighborsClassifier(n_neighbors=k,
                               weights='distance').fit(X_train, y_train)

    predictions = knn.predict(X_test)

    acc = accuracy_score(y_test, predictions)
    prfs = np.vstack(precision_recall_fscore_support(predictions, y_test))

    print('Overall accuracy: {:f}'.format(acc))
    print('')
    print(pd.DataFrame(data=prfs,
                       index=['Precision', 'Recall', 'F-1', 'Support'],
                       columns=knn.classes_))

    if preds_file:
        np.savetxt(preds_file, predictions, delimiter=' ', fmt='%s')

    return acc, prfs